In [1]:
import pygsp
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from caveclient import CAVEclient
import torch
from torch_geometric import utils
from torch_geometric.data import Data
from tqdm import tqdm 
from pygod.models import DOMINANT

In [2]:
#import functions 
from MinnieData_Table_Extraction import extract_full_nodes, get_nodes_table, set_node_attributes
from Node_Attributes import allens_prediction,gac_prediction, in_out_degs, cell_type

In [ ]:
def filter_by_layer_connected(nodes, edges, layer):
    '''
    Extract subgraphs from June connectome data by layer for analysis
    Parameters: 
        nodes (Pandas dataframe) final june node dataframe
        edges (Pandas dataframe) final june edge dataframe 
        layer (string) layer of interest to extract
    Return: 
        node table subgraph (Pandas dataframe) by layer without orphans
        edge table subgraph (Pandas dataframe) by layer
        Subgraph G (NetworkX graph object) without orphans
    '''

    # Filter entire node table to only those in layer 
    rslt_synapse_df = nodes[nodes['layer'] == layer] 

    # Get list of unique neurons
    list_of_neurons = set(rslt_synapse_df['pt_root_id'].to_list())
    # make df with list of neurons FIX IT

    # Filter the synapse tables  extract all unique connections between nodes
    filtered = edges[edges['Source'].isin(list_of_neurons)]
    filtered2 = filtered[filtered['Target'].isin(list_of_neurons)]

    # create fully connected graph object
    G = nx.from_pandas_edgelist(filtered2, source="Source", target="Target", create_using=nx.DiGraph, edge_attr=["Euclidean_Distance"])
    list_of_current_nodes = set(list(G.nodes))

    # return node_df for fully connected graph object (does not include orphans)
    connected_node_list = list(G.nodes())
    rslt_synapse_df = rslt_synapse_df[rslt_synapse_df['pt_root_id'].isin(connected_node_list)]

    #reformatting node_df and dropping duplicates
    rslt_synapse_df = rslt_synapse_df.reset_index()
    rslt_synapse_df = rslt_synapse_df.drop(columns=['index','Unnamed: 0'], axis=1)
    rslt_synapse_df = rslt_synapse_df.drop_duplicates(subset=['pt_root_id'])
    
    return rslt_synapse_df, filtered2, G


In [3]:
#load in node and edge list (from extract_full_nodes)
nodes = pd.read_csv("final_connectome_node_table.csv") 
edges = pd.read_csv("final_connectome_edge_table.csv")

## Layer 1

In [4]:
#Extract node table, edge table, and graph object (No orphans in node table or graph)
node_df_L1, edge_df_L1, G_L1 = filter_by_layer_connected(nodes, edges, 'L1')  

In [5]:
node_df_L1

,id,pt_root_id,pt_position,timestamp,layer,cell_polarity,subclass,pre_syn_count,post_syn_count,unique_pre_syn_target_count,unique_post_syn_target_count
0,485509,864691136740606812,[282608 103808 20318],2022-06-09 14:30:32.002707,L1,excitatory,23P,93.0,6802.0,77.0,6133.0
1,420548,864691135463500869,[261040 103568 19671],2022-06-09 14:30:32.002707,L1,excitatory,23P,602.0,4492.0,480.0,3801.0
2,484868,864691136199051157,[290608 84320 24817],2022-06-09 14:30:32.002707,L1,excitatory,23P,235.0,2105.0,216.0,1984.0
3,104850,864691136311834173,[105328 109056 20813],2022-06-09 14:30:32.002707,L1,excitatory,23P,42.0,3350.0,31.0,3046.0
4,420304,864691135567738604,[251200 102352 18621],2022-06-09 14:30:32.002707,L1,excitatory,23P,80.0,4588.0,65.0,4192.0
...,...,...,...,...,...,...,...,...,...,...,...
1284,420328,864691135501873474,[256880 101424 18112],2022-06-09 14:30:32.002707,L1,excitatory,23P,129.0,4810.0,110.0,4445.0
1285,607791,864691135771586555,[348368 97248 15497],2022-06-09 14:30:32.002707,L1,inhibitory,BPC,13.0,894.0,10.0,778.0
1286,290914,864691135194551082,[193296 106240 18412],2022-06-09 14:30:32.002707,L1,inhibitory,NGC,2024.0,3424.0,1626.0,3190.0
1287,607592,864691135354901967,[360224 86496 25046],2022-06-09 14:30:32.002707,L1,excitatory,23P,21.0,2760.0,15.0,2630.0


In [6]:
#one hot encode cell_polarity (excitatory vs inhibitory) and subclass (cell type)
node_df_hot_L1 = pd.get_dummies(node_df_L1, columns=["subclass","cell_polarity"], prefix={"Cell Type","Cell Classification"})
node_df_hot_L1 = node_df_hot_L1.drop(['id','timestamp','unique_pre_syn_target_count','unique_post_syn_target_count'],axis=1)
#Relabel pre_synaptic_count as out degree and vice versa
node_df_hot_L1 = node_df_hot_L1.rename(columns={'pre_syn_count':'out degree','post_syn_count':'in degree'}) 

In [7]:
#checking for unique cell types to set node attributes 
node_df_L1['subclass'].unique()

array(['23P', 'NGC', 'MC', 'BPC', 'BC', '5P-PT', '4P'], dtype=object)

In [8]:
node_attributes_L1=['in degree','out degree',
        'Cell Type_23P', 'Cell Type_4P','Cell Type_5P-PT',
        'Cell Type_BC', 'Cell Type_BPC','Cell Type_MC','Cell Type_NGC',
        'Cell Classification_excitatory','Cell Classification_inhibitory']

In [9]:
#set node attributes 
for index, row in node_df_hot_L1.iterrows():
     node_attr_dict_L1={k: float(row.to_dict()[k]) for k in node_attributes_L1}
     G_L1.nodes[row['pt_root_id']].update(node_attr_dict_L1)

In [10]:
#create py geometric data object
Data_L1=utils.from_networkx(G_L1, group_node_attrs=('in degree','out degree',
        'Cell Type_23P', 'Cell Type_4P','Cell Type_5P-PT',
        'Cell Type_BC', 'Cell Type_BPC','Cell Type_MC','Cell Type_NGC',
        'Cell Classification_excitatory','Cell Classification_inhibitory'
        ))

In [11]:
#run DOMINANT
model_DOM_L1 = DOMINANT()
model_DOM_L1.fit(Data_L1)

labels_DOM_L1 = model_DOM_L1.predict(Data_L1)
print('Labels:')
print(labels_DOM_L1)

outlier_scores_DOM_L1 = model_DOM_L1.decision_function(Data_L1)
print('Raw scores:')
print(outlier_scores_DOM_L1)

prob_DOM_L1 = model_DOM_L1.predict_proba(Data_L1)
print('Probability:')
print(prob_DOM_L1)

label_DOM_L1, confidence_DOM_L1 = model_DOM_L1.predict(Data_L1, return_confidence=True)
print('Labels:')
print(labels_DOM_L1)
print('Confidence:')
print(confidence_DOM_L1)

/Users/vivialung/CRIMSON Coding/madan_test/lib/python3.7/site-packages/pygod/utils/utility.py:52: UserWarning: The cuda is not available. Set to cpu.
  warnings.warn('The cuda is not available. Set to cpu.')


Labels:
[0 0 0 ... 0 0 0]
Raw scores:
[41273648. 55655452. 81340992. ... 26097324. 36335228.  4347689.]
Probability:
[[0.94082026 0.05917974]
 [0.91966661 0.08033339]
 [0.88188673 0.11811327]
 ...
 [0.96314254 0.03685746]
 [0.94808399 0.05191601]
 [0.99513325 0.00486675]]
Labels:
[0 0 0 ... 0 0 0]
Confidence:
[1. 1. 1. ... 1. 1. 1.]


In [12]:
print(G_L1)

DiGraph with 1288 nodes and 10119 edges


In [13]:
#number of anomalous nodes labeled by DOMINANT
np.sum(labels_DOM_L1)

123

In [14]:
#Grabbing seg_ids of anomalous nodes labeled by DOMINANT
DOMINANT_L1_AD = node_df_L1.copy()
DOMINANT_L1_AD['DOM_labels'] = labels_DOM_L1
DOMINANT_L1_AD = DOMINANT_L1_AD[DOMINANT_L1_AD['DOM_labels'] == 1]

In [15]:
DOMINANT_L1_AD

,id,pt_root_id,pt_position,timestamp,layer,cell_polarity,subclass,pre_syn_count,post_syn_count,unique_pre_syn_target_count,unique_post_syn_target_count,DOM_labels
4,420304,864691135567738604,[251200 102352 18621],2022-06-09 14:30:32.002707,L1,excitatory,23P,80.0,4588.0,65.0,4192.0,1
6,516026,864691136541636514,[303952 92304 21439],2022-06-09 14:30:32.002707,L1,inhibitory,NGC,151.0,3280.0,141.0,3089.0,1
8,284044,864691135500679108,[181584 93536 15536],2022-06-09 14:30:32.002707,L1,inhibitory,NGC,178.0,2483.0,158.0,2347.0,1
17,485329,864691136273644301,[279648 100240 17581],2022-06-09 14:30:32.002707,L1,excitatory,23P,137.0,5819.0,116.0,5270.0,1
20,255318,864691135581550701,[174016 97872 23685],2022-06-09 14:30:32.002707,L1,excitatory,23P,56.0,4059.0,49.0,3765.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
816,580166,864691136617102555,[330096 88704 22534],2022-06-09 14:30:32.002707,L1,excitatory,23P,26.0,2575.0,13.0,2378.0,1
883,516326,864691136773850734,[300096 84864 26327],2022-06-09 14:30:32.002707,L1,inhibitory,MC,187.0,2886.0,160.0,2738.0,1
931,484838,864691135809517004,[282112 86576 24497],2022-06-09 14:30:32.002707,L1,excitatory,23P,129.0,5191.0,108.0,4613.0,1
962,319422,864691135385187669,[198576 91488 16630],2022-06-09 14:30:32.002707,L1,inhibitory,NGC,84.0,2077.0,67.0,1969.0,1


In [16]:
#finding 80th percentile (top 20% of anomalous nodes)
percentile80_L1 = np.percentile(prob_DOM_L1[:,1], 80)
L1_AD_indarray = prob_DOM_L1[:,1] > percentile80_L1
DOMINANT_L1_AD_80 = node_df_L1.iloc[L1_AD_indarray]

In [17]:
#DOMINANT_L1_AD_80.to_csv('DOMINANT_L!_AD_80.csv')

In [110]:
#DOMINANT_L1_AD.to_csv('DOMINANT_L1_AD.csv')

## Layers 23

In [4]:
#Extract node table, edge table, and graph object (No orphans in node table or graph)
node_df_L23, edge_df_L23, G_L23 = filter_by_layer_connected(nodes, edges, 'L23')  # sample run for subgraph L23

In [5]:
node_df_L23

,id,pt_root_id,pt_position,timestamp,layer,cell_polarity,subclass,pre_syn_count,post_syn_count,unique_pre_syn_target_count,unique_post_syn_target_count
0,223131,864691135695974042,[150880 111632 24610],2022-06-09 14:30:32.002707,L23,excitatory,23P,260.0,4928.0,236.0,4339.0
1,361213,864691135416006330,[217072 132656 19736],2022-06-09 14:30:32.002707,L23,excitatory,23P,222.0,3591.0,183.0,3211.0
2,518339,864691135733291156,[305536 121200 20212],2022-06-09 14:30:32.002707,L23,excitatory,23P,41.0,3921.0,31.0,3624.0
3,517979,864691135974854346,[295200 123808 17670],2022-06-09 14:30:32.002707,L23,excitatory,23P,133.0,6305.0,93.0,5375.0
4,358680,864691135478435270,[217840 122528 16234],2022-06-09 14:30:32.002707,L23,excitatory,23P,309.0,3863.0,261.0,3534.0
...,...,...,...,...,...,...,...,...,...,...,...
14063,256117,864691135394016245,[167664 126320 19345],2022-06-09 14:30:32.002707,L23,excitatory,23P,60.0,2540.0,56.0,2398.0
14064,420258,864691135403724270,[254560 107232 17475],2022-06-09 14:30:32.002707,L23,excitatory,23P,46.0,4481.0,39.0,4109.0
14065,390522,864691135571211757,[239920 112864 17348],2022-06-09 14:30:32.002707,L23,excitatory,23P,86.0,4757.0,68.0,4375.0
14066,256280,864691136966016974,[165808 118256 20198],2022-06-09 14:30:32.002707,L23,excitatory,23P,782.0,5362.0,586.0,4844.0


In [8]:
#one hot encode cell_polarity (excitatory vs inhibitory) and subclass (cell type)
node_df_hot_L23 = pd.get_dummies(node_df_L23, columns=["subclass","cell_polarity"], prefix={"Cell Type","Cell Classification"})
node_df_hot_L23 = node_df_hot_L23.drop(['id','timestamp','unique_pre_syn_target_count','unique_post_syn_target_count'],axis=1)
#Relabel pre_synaptic_count as out degree and vice versa
node_df_hot_L23 = node_df_hot_L23.rename(columns={'pre_syn_count':'out degree','post_syn_count':'in degree'}) #Relabel pre_synaptic_count as out degree and vice versa
#checking for unique cell types to set node attributes 
node_df_L23['subclass'].unique()


array(['23P', 'BPC', 'BC', 'NGC', 'MC', '4P', '5P-PT'], dtype=object)

In [9]:
node_attributes_L23=['in degree','out degree',
        'Cell Type_23P', 'Cell Type_4P','Cell Type_5P-PT',
        'Cell Type_BC', 'Cell Type_BPC','Cell Type_MC', 'Cell Type_NGC',
        'Cell Classification_excitatory','Cell Classification_inhibitory']

#set node attributes 
for index, row in node_df_hot_L23.iterrows():
     node_attr_dict_L23={k: float(row.to_dict()[k]) for k in node_attributes_L23}
     G_L23.nodes[row['pt_root_id']].update(node_attr_dict_L23)


In [10]:
#creating py geometric data object
Data_L23=utils.from_networkx(G_L23, group_node_attrs=('in degree','out degree',
        'Cell Type_23P', 'Cell Type_4P','Cell Type_5P-PT',
        'Cell Type_BC', 'Cell Type_BPC','Cell Type_MC','Cell Type_NGC',
        'Cell Classification_excitatory','Cell Classification_inhibitory'
        ))

In [12]:
#running DOMINANT
model_DOM_L23 = DOMINANT()
model_DOM_L23.fit(Data_L23)

labels_DOM_L23 = model_DOM_L23.predict(Data_L23)
print('Labels:')
print(labels_DOM_L23)

outlier_scores_DOM_L23 = model_DOM_L23.decision_function(Data_L23)
print('Raw scores:')
print(outlier_scores_DOM_L23)

prob_DOM_L23 = model_DOM_L23.predict_proba(Data_L23)
print('Probability:')
print(prob_DOM_L23)

label_DOM_L23, confidence_DOM_L23 = model_DOM_L23.predict(Data_L23, return_confidence=True)
print('Labels:')
print(labels_DOM_L23)
print('Confidence:')
print(confidence_DOM_L23)

/Users/vivialung/CRIMSON Coding/madan_test/lib/python3.7/site-packages/pygod/utils/utility.py:52: UserWarning: The cuda is not available. Set to cpu.
  warnings.warn('The cuda is not available. Set to cpu.')


Labels:
[0 0 1 ... 0 0 0]
Raw scores:
[3.35999744e+08 2.03691648e+08 1.67180774e+09 ... 1.76440020e+07
 3.39819050e+06 6.48403650e+06]
Probability:
[[0.84541986 0.15458014]
 [0.90644312 0.09355688]
 [0.22931722 0.77068278]
 ...
 [0.99225219 0.00774781]
 [0.99882266 0.00117734]
 [0.9973994  0.0026006 ]]
Labels:
[0 0 1 ... 0 0 0]
Confidence:
[1. 1. 1. ... 1. 1. 1.]


In [85]:
print(G_L23)

DiGraph with 14049 nodes and 767863 edges


In [13]:
#number of anomalous nodes labeled by DOMINANT
np.sum(labels_DOM_L23)

1919

In [111]:
#Grabbing seg_ids of anomalous nodes labeled by DOMINANT
DOMINANT_L23_AD = node_df_L23.copy()
DOMINANT_L23_AD['DOM_labels'] = labels_DOM_L23
DOMINANT_L23_AD = DOMINANT_L23_AD[DOMINANT_L23_AD['DOM_labels'] == 1]

In [17]:
#finding 80th percentile (top 20% of anomalous nodes)
percentile80_L23 = np.percentile(prob_DOM_L23[:,1], 80)
L23_AD_indarray = prob_DOM_L23[:,1] > percentile80_L23
DOMINANT_L23_AD_80 = node_df_L23.iloc[L23_AD_indarray]
DOMINANT_L23_AD_80

,id,pt_root_id,pt_position,timestamp,layer,cell_polarity,subclass,pre_syn_count,post_syn_count,unique_pre_syn_target_count,unique_post_syn_target_count
2,518339,864691135733291156,[305536 121200 20212],2022-06-09 14:30:32.002707,L23,excitatory,23P,41.0,3921.0,31.0,3624.0
3,517979,864691135974854346,[295200 123808 17670],2022-06-09 14:30:32.002707,L23,excitatory,23P,133.0,6305.0,93.0,5375.0
5,360132,864691135396729889,[216112 122000 26320],2022-06-09 14:30:32.002707,L23,excitatory,23P,108.0,2527.0,83.0,2326.0
6,610723,864691135544615976,[345568 110960 21799],2022-06-09 14:30:32.002707,L23,excitatory,23P,52.0,4023.0,41.0,3652.0
17,106826,864691136039346686,[107392 117232 20335],2022-06-09 14:30:32.002707,L23,excitatory,23P,187.0,2956.0,163.0,2623.0
...,...,...,...,...,...,...,...,...,...,...,...
13142,358273,864691135338090982,[221216 100224 25583],2022-06-09 14:30:32.002707,L23,excitatory,23P,177.0,3552.0,152.0,3282.0
13159,63734,864691136123784742,[ 89360 121936 16971],2022-06-09 14:30:32.002707,L23,excitatory,23P,235.0,1738.0,203.0,1647.0
13203,358554,864691136682605174,[213376 120720 15576],2022-06-09 14:30:32.002707,L23,excitatory,23P,71.0,3345.0,58.0,3083.0
13350,109446,864691136436404638,[105136 130768 18192],2022-06-09 14:30:32.002707,L23,excitatory,23P,72.0,2747.0,57.0,2523.0


In [113]:
#DOMINANT_L23_AD.to_csv('DOMINANT_L23_AD.csv')

## Layer 4 

In [35]:
#Extract node table, edge table, and graph object (No orphans in node table or graph)
node_df_L4, edge_df_L4, G_L4 = filter_by_layer_connected(nodes, edges, 'L4')  # sample run for subgraph L23

#one hot encode cell_polarity (excitatory vs inhibitory) and subclass (cell type)
node_df_hot_L4 = pd.get_dummies(node_df_L4, columns=["subclass","cell_polarity"], prefix={"Cell Type","Cell Classification"})
node_df_hot_L4 = node_df_hot_L4.drop(['id','timestamp','unique_pre_syn_target_count','unique_post_syn_target_count'],axis=1)
#Relabel pre_synaptic_count as out degree and vice versa
node_df_hot_L4 = node_df_hot_L4.rename(columns={'pre_syn_count':'out degree','post_syn_count':'in degree'}) #Relabel pre_synaptic_count as out degree and vice versa
#checking for unique cell types to set node attributes 
node_df_L4['subclass'].unique()

array(['4P', '23P', 'MC', 'BC', '5P-PT', '5P-IT', 'BPC', 'NGC'],
      dtype=object)

In [37]:
node_attributes_L4=['in degree','out degree',
        'Cell Type_23P', 'Cell Type_4P','Cell Type_5P-PT','Cell Type_5P-IT',
        'Cell Type_BC', 'Cell Type_BPC','Cell Type_MC', 'Cell Type_NGC',
        'Cell Classification_excitatory','Cell Classification_inhibitory']

#set node attributes 
for index, row in node_df_hot_L4.iterrows():
     node_attr_dict_L4={k: float(row.to_dict()[k]) for k in node_attributes_L4}
     G_L4.nodes[row['pt_root_id']].update(node_attr_dict_L4)

#creating py geometric data object
Data_L4=utils.from_networkx(G_L4, group_node_attrs=('in degree','out degree',
        'Cell Type_23P', 'Cell Type_4P','Cell Type_5P-PT', 'Cell Type_5P-IT',
        'Cell Type_BC', 'Cell Type_BPC','Cell Type_MC','Cell Type_NGC',
        'Cell Classification_excitatory','Cell Classification_inhibitory'
        ))

In [38]:
#running DOMINANT
model_DOM_L4 = DOMINANT()
model_DOM_L4.fit(Data_L4)

labels_DOM_L4 = model_DOM_L4.predict(Data_L4)
print('Labels:')
print(labels_DOM_L4)

outlier_scores_DOM_L4 = model_DOM_L4.decision_function(Data_L4)
print('Raw scores:')
print(outlier_scores_DOM_L4)

prob_DOM_L4 = model_DOM_L4.predict_proba(Data_L4)
print('Probability:')
print(prob_DOM_L4)

label_DOM_L4, confidence_DOM_L4 = model_DOM_L4.predict(Data_L4, return_confidence=True)
print('Labels:')
print(labels_DOM_L4)
print('Confidence:')
print(confidence_DOM_L4)

/Users/vivialung/CRIMSON Coding/madan_test/lib/python3.7/site-packages/pygod/utils/utility.py:52: UserWarning: The cuda is not available. Set to cpu.
  warnings.warn('The cuda is not available. Set to cpu.')


Labels:
[0 0 0 ... 0 0 0]
Raw scores:
[2.76919240e+07 8.36614400e+07 1.51579728e+08 ... 1.88526580e+07
 7.01115100e+06 1.62064690e+07]
Probability:
[[0.97062235 0.02937765]
 [0.91072368 0.08927632]
 [0.83803742 0.16196258]
 ...
 [0.98008215 0.01991785]
 [0.99275495 0.00724505]
 [0.9829141  0.0170859 ]]
Labels:
[0 0 0 ... 0 0 0]
Confidence:
[1. 1. 1. ... 1. 1. 1.]


In [41]:
print(G_L4)

DiGraph with 17335 nodes and 1194054 edges


In [42]:
#number of anomalous nodes labeled by DOMINANT
np.sum(labels_DOM_L4)

3999

In [114]:
#Grabbing seg_ids of anomalous nodes labeled by DOMINANT
DOMINANT_L4_AD = node_df_L4.copy()
DOMINANT_L4_AD['DOM_labels'] = labels_DOM_L4
DOMINANT_L4_AD = DOMINANT_L4_AD[DOMINANT_L4_AD['DOM_labels'] == 1]

In [116]:
#finding 80th percentile (top 20% of anomalous nodes)
percentile80_L4 = np.percentile(prob_DOM_L4[:,1], 80)
L4_AD_indarray = prob_DOM_L4[:,1] > percentile80_L4
DOMINANT_L4_AD_80 = node_df_L4.iloc[L4_AD_indarray]
DOMINANT_L4_AD_80

,id,pt_root_id,pt_position,timestamp,layer,cell_polarity,subclass,pre_syn_count,post_syn_count,unique_pre_syn_target_count,unique_post_syn_target_count
7,612818,864691135965599690,[344064 132688 23873],2022-06-09 14:30:32.002707,L4,excitatory,4P,266.0,5310.0,230.0,4580.0
11,261392,864691135883891312,[178128 157872 27288],2022-06-09 14:30:32.002707,L4,excitatory,4P,109.0,2501.0,88.0,2149.0
14,113155,864691135809476044,[101552 149616 21520],2022-06-09 14:30:32.002707,L4,excitatory,23P,48.0,561.0,44.0,505.0
21,394887,864691135939866534,[237744 143680 22129],2022-06-09 14:30:32.002707,L4,excitatory,4P,77.0,3172.0,64.0,2667.0
22,490967,864691135715236378,[285952 145968 20862],2022-06-09 14:30:32.002707,L4,excitatory,4P,328.0,2914.0,282.0,2508.0
...,...,...,...,...,...,...,...,...,...,...,...
16535,295572,864691135334551529,[182976 150800 15916],2022-06-09 14:30:32.002707,L4,excitatory,4P,415.0,3252.0,355.0,2808.0
16752,520676,864691135132673440,[302160 131408 24294],2022-06-09 14:30:32.002707,L4,excitatory,23P,294.0,5427.0,259.0,4753.0
16847,396998,864691135946642785,[233088 160608 22847],2022-06-09 14:30:32.002707,L4,excitatory,4P,338.0,2148.0,295.0,1869.0
17050,73529,864691135538423666,[ 90592 171952 17150],2022-06-09 14:30:32.002707,L4,excitatory,4P,181.0,1096.0,159.0,951.0


In [117]:
#DOMINANT_L4_AD.to_csv('DOMINANT_L4_AD.csv')

## Layer 5

In [45]:
#Extract node table, edge table, and graph object (No orphans in node table or graph)
node_df_L5, edge_df_L5, G_L5 = filter_by_layer_connected(nodes, edges, 'L5')  # sample run for subgraph L23

#one hot encode cell_polarity (excitatory vs inhibitory) and subclass (cell type)
node_df_hot_L5 = pd.get_dummies(node_df_L5, columns=["subclass","cell_polarity"], prefix={"Cell Type","Cell Classification"})
node_df_hot_L5 = node_df_hot_L5.drop(['id','timestamp','unique_pre_syn_target_count','unique_post_syn_target_count'],axis=1)
#Relabel pre_synaptic_count as out degree and vice versa
node_df_hot_L5 = node_df_hot_L5.rename(columns={'pre_syn_count':'out degree','post_syn_count':'in degree'}) #Relabel pre_synaptic_count as out degree and vice versa
#checking for unique cell types to set node attributes 
node_df_L5['subclass'].unique()

array(['5P-IT', '4P', '5P-PT', 'BC', '6P-IT', 'MC', '5P-NP', '6P-CT',
       'NGC', '23P', 'BPC'], dtype=object)

In [52]:
node_attributes_L5=['in degree','out degree',
        'Cell Type_23P', 'Cell Type_4P','Cell Type_5P-PT','Cell Type_5P-IT', 'Cell Type_6P-IT', 'Cell Type_5P-NP','Cell Type_6P-CT',
        'Cell Type_BC', 'Cell Type_BPC','Cell Type_MC', 'Cell Type_NGC',
        'Cell Classification_excitatory','Cell Classification_inhibitory']

#set node attributes 
for index, row in node_df_hot_L5.iterrows():
     node_attr_dict_L5={k: float(row.to_dict()[k]) for k in node_attributes_L5}
     G_L5.nodes[row['pt_root_id']].update(node_attr_dict_L5)

#creating py geometric data object
Data_L5=utils.from_networkx(G_L5, group_node_attrs=('in degree','out degree',
        'Cell Type_23P', 'Cell Type_4P','Cell Type_5P-PT', 'Cell Type_5P-IT', 'Cell Type_6P-IT', 'Cell Type_5P-NP','Cell Type_6P-CT',
        'Cell Type_BC', 'Cell Type_BPC','Cell Type_MC','Cell Type_NGC',
        'Cell Classification_excitatory','Cell Classification_inhibitory'
        ))

In [53]:
#running DOMINANT
model_DOM_L5 = DOMINANT()
model_DOM_L5.fit(Data_L5)

labels_DOM_L5 = model_DOM_L5.predict(Data_L5)
print('Labels:')
print(labels_DOM_L5)

outlier_scores_DOM_L5 = model_DOM_L5.decision_function(Data_L5)
print('Raw scores:')
print(outlier_scores_DOM_L5)

prob_DOM_L5 = model_DOM_L5.predict_proba(Data_L5)
print('Probability:')
print(prob_DOM_L5)

label_DOM_L5, confidence_DOM_L5 = model_DOM_L5.predict(Data_L5, return_confidence=True)
print('Labels:')
print(labels_DOM_L5)
print('Confidence:')
print(confidence_DOM_L5)

/Users/vivialung/CRIMSON Coding/madan_test/lib/python3.7/site-packages/pygod/utils/utility.py:52: UserWarning: The cuda is not available. Set to cpu.
  warnings.warn('The cuda is not available. Set to cpu.')


Labels:
[0 0 0 ... 0 0 0]
Raw scores:
[3.33134368e+08 4.60866912e+08 1.79993760e+08 ... 4.08808200e+07
 1.63228450e+06 1.24361930e+07]
Probability:
[[8.24817660e-01 1.75182340e-01]
 [7.57607350e-01 2.42392650e-01]
 [9.05397186e-01 9.46028144e-02]
 ...
 [9.78595632e-01 2.14043676e-02]
 [9.99247427e-01 7.52573212e-04]
 [9.93562626e-01 6.43737379e-03]]
Labels:
[0 0 0 ... 0 0 0]
Confidence:
[1. 1. 1. ... 1. 1. 1.]


In [54]:
print(G_L5)

DiGraph with 13436 nodes and 1212430 edges


In [55]:
#number of anomalous nodes labeled by DOMINANT
np.sum(labels_DOM_L5)

1330

In [119]:
#Grabbing seg_ids of anomalous nodes labeled by DOMINANT
DOMINANT_L5_AD = node_df_L5.copy()
DOMINANT_L5_AD['DOM_labels'] = labels_DOM_L5
DOMINANT_L5_AD = DOMINANT_L5_AD[DOMINANT_L5_AD['DOM_labels'] == 1]

In [120]:
#finding 80th percentile (top 20% of anomalous nodes)
percentile80_L5 = np.percentile(prob_DOM_L5[:,1], 80)
L5_AD_indarray = prob_DOM_L5[:,1] > percentile80_L5
DOMINANT_L5_AD_80 = node_df_L5.iloc[L5_AD_indarray]
DOMINANT_L5_AD_80

,id,pt_root_id,pt_position,timestamp,layer,cell_polarity,subclass,pre_syn_count,post_syn_count,unique_pre_syn_target_count,unique_post_syn_target_count
1,302377,864691135502005173,[180624 193904 17703],2022-06-09 14:30:32.002707,L5,excitatory,5P-IT,295.0,2597.0,248.0,2333.0
4,495262,864691135991244746,[284848 187728 21257],2022-06-09 14:30:32.002707,L5,excitatory,5P-IT,225.0,3145.0,193.0,2829.0
10,493008,864691135476436136,[288320 174128 19656],2022-06-09 14:30:32.002707,L5,excitatory,5P-IT,182.0,3486.0,147.0,3091.0
16,372466,864691135341064645,[221552 212064 24280],2022-06-09 14:30:32.002707,L5,excitatory,6P-IT,30.0,4224.0,19.0,3566.0
24,462627,864691136913511534,[274128 191568 25437],2022-06-09 14:30:32.002707,L5,excitatory,5P-PT,616.0,6760.0,465.0,5530.0
...,...,...,...,...,...,...,...,...,...,...,...
12149,396601,864691135946966113,[238080 167824 20031],2022-06-09 14:30:32.002707,L5,inhibitory,BPC,94.0,5759.0,77.0,5195.0
12208,263935,864691136051137779,[176560 187088 16388],2022-06-09 14:30:32.002707,L5,excitatory,5P-NP,138.0,1271.0,122.0,1188.0
12223,558248,864691135445914770,[314624 173360 16639],2022-06-09 14:30:32.002707,L5,excitatory,5P-IT,77.0,2230.0,63.0,1912.0
12318,197928,864691135988722944,[143520 180464 27207],2022-06-09 14:30:32.002707,L5,excitatory,5P-IT,159.0,1225.0,136.0,1035.0


In [121]:
#DOMINANT_L5_AD.to_csv('DOMINANT_L5_AD.csv')

## Layer 6

In [57]:
#Extract node table, edge table, and graph object (No orphans in node table or graph)
node_df_L6, edge_df_L6, G_L6 = filter_by_layer_connected(nodes, edges, 'L6')  # sample run for subgraph L23

#one hot encode cell_polarity (excitatory vs inhibitory) and subclass (cell type)
node_df_hot_L6 = pd.get_dummies(node_df_L6, columns=["subclass","cell_polarity"], prefix={"Cell Type","Cell Classification"})
node_df_hot_L6 = node_df_hot_L6.drop(['id','timestamp','unique_pre_syn_target_count','unique_post_syn_target_count'],axis=1)
#Relabel pre_synaptic_count as out degree and vice versa
node_df_hot_L6 = node_df_hot_L6.rename(columns={'pre_syn_count':'out degree','post_syn_count':'in degree'}) #Relabel pre_synaptic_count as out degree and vice versa
#checking for unique cell types to set node attributes 
node_df_L6['subclass'].unique()

array(['6P-IT', '5P-IT', 'MC', '6P-CT', '5P-NP', 'BC', 'NGC', 'BPC',
       '5P-PT', '23P', '4P'], dtype=object)

In [58]:
node_attributes_L6=['in degree','out degree',
        'Cell Type_23P', 'Cell Type_4P','Cell Type_5P-PT','Cell Type_5P-IT', 'Cell Type_6P-IT', 'Cell Type_5P-NP','Cell Type_6P-CT',
        'Cell Type_BC', 'Cell Type_BPC','Cell Type_MC', 'Cell Type_NGC',
        'Cell Classification_excitatory','Cell Classification_inhibitory']

#set node attributes 
for index, row in node_df_hot_L6.iterrows():
     node_attr_dict_L6={k: float(row.to_dict()[k]) for k in node_attributes_L6}
     G_L6.nodes[row['pt_root_id']].update(node_attr_dict_L6)

#creating py geometric data object
Data_L6=utils.from_networkx(G_L6, group_node_attrs=('in degree','out degree',
        'Cell Type_23P', 'Cell Type_4P','Cell Type_5P-PT', 'Cell Type_5P-IT', 'Cell Type_6P-IT', 'Cell Type_5P-NP','Cell Type_6P-CT',
        'Cell Type_BC', 'Cell Type_BPC','Cell Type_MC','Cell Type_NGC',
        'Cell Classification_excitatory','Cell Classification_inhibitory'
        ))

In [59]:
#running DOMINANT
model_DOM_L6 = DOMINANT()
model_DOM_L6.fit(Data_L6)

labels_DOM_L6 = model_DOM_L6.predict(Data_L6)
print('Labels:')
print(labels_DOM_L6)

outlier_scores_DOM_L6 = model_DOM_L6.decision_function(Data_L6)
print('Raw scores:')
print(outlier_scores_DOM_L6)

prob_DOM_L6 = model_DOM_L6.predict_proba(Data_L6)
print('Probability:')
print(prob_DOM_L6)

label_DOM_L6, confidence_DOM_L6 = model_DOM_L6.predict(Data_L6, return_confidence=True)
print('Labels:')
print(labels_DOM_L6)
print('Confidence:')
print(confidence_DOM_L6)

/Users/vivialung/CRIMSON Coding/madan_test/lib/python3.7/site-packages/pygod/utils/utility.py:52: UserWarning: The cuda is not available. Set to cpu.
  warnings.warn('The cuda is not available. Set to cpu.')


Labels:
[1 0 0 ... 0 0 0]
Raw scores:
[4.29056256e+08 7.07946160e+07 5.73750360e+07 ... 1.12995550e+06
 5.66536750e+06 1.04900238e+06]
Probability:
[[0.47048658 0.52951342]
 [0.91267326 0.08732674]
 [0.92923646 0.07076354]
 ...
 [0.99865731 0.00134269]
 [0.99305945 0.00694055]
 [0.99875723 0.00124277]]
Labels:
[1 0 0 ... 0 0 0]
Confidence:
[1. 1. 1. ... 1. 1. 1.]


In [60]:
print(G_L6)

DiGraph with 18620 nodes and 1023028 edges


In [61]:
#number of anomalous nodes labeled by DOMINANT
np.sum(labels_DOM_L6)

1747

In [122]:
#Grabbing seg_ids of anomalous nodes labeled by DOMINANT
DOMINANT_L6_AD = node_df_L6.copy()
DOMINANT_L6_AD['DOM_labels'] = labels_DOM_L6
DOMINANT_L6_AD = DOMINANT_L6_AD[DOMINANT_L6_AD['DOM_labels'] == 1]

In [124]:
#finding 80th percentile (top 20% of anomalous nodes)
percentile80_L6 = np.percentile(prob_DOM_L6[:,1], 80)
L6_AD_indarray = prob_DOM_L6[:,1] > percentile80_L6
DOMINANT_L6_AD_80 = node_df_L6.iloc[L6_AD_indarray]
DOMINANT_L6_AD_80

,id,pt_root_id,pt_position,timestamp,layer,cell_polarity,subclass,pre_syn_count,post_syn_count,unique_pre_syn_target_count,unique_post_syn_target_count
0,204710,864691135497604883,[134208 247184 18598],2022-06-09 14:30:32.002707,L6,excitatory,6P-IT,111.0,2125.0,95.0,1924.0
10,273184,864691135454040938,[163888 244368 18219],2022-06-09 14:30:32.002707,L6,excitatory,6P-IT,40.0,2106.0,28.0,1936.0
11,717053,864691134988430970,[390144 222720 23159],2022-06-09 14:30:32.002707,L6,excitatory,6P-IT,117.0,714.0,99.0,647.0
12,719779,864691136116277924,[378256 240304 19202],2022-06-09 14:30:32.002707,L6,excitatory,6P-IT,15.0,118.0,9.0,89.0
15,202680,864691135207767417,[143184 238464 17307],2022-06-09 14:30:32.002707,L6,excitatory,6P-IT,129.0,1878.0,100.0,1723.0
...,...,...,...,...,...,...,...,...,...,...,...
16808,500493,864691135761550902,[290496 230144 17726],2022-06-09 14:30:32.002707,L6,excitatory,6P-IT,76.0,2153.0,53.0,1987.0
16973,624284,864691135730202553,[347488 228336 24857],2022-06-09 14:30:32.002707,L6,excitatory,6P-CT,91.0,1401.0,78.0,1278.0
17054,201705,864691135593520427,[142240 221504 24422],2022-06-09 14:30:32.002707,L6,excitatory,6P-CT,266.0,2162.0,217.0,1904.0
17128,592450,864691135345129119,[342528 220800 18699],2022-06-09 14:30:32.002707,L6,excitatory,6P-IT,47.0,2131.0,36.0,1990.0


In [125]:
#DOMINANT_L6_AD.to_csv('DOMINANT_L6_AD.csv')

## WM

In [63]:
#Extract node table, edge table, and graph object (No orphans in node table or graph)
node_df_WM, edge_df_WM, G_WM = filter_by_layer_connected(nodes, edges, 'WM')  # sample run for subgraph L23

#one hot encode cell_polarity (excitatory vs inhibitory) and subclass (cell type)
node_df_hot_WM = pd.get_dummies(node_df_WM, columns=["subclass","cell_polarity"], prefix={"Cell Type","Cell Classification"})
node_df_hot_WM = node_df_hot_WM.drop(['id','timestamp','unique_pre_syn_target_count','unique_post_syn_target_count'],axis=1)
node_df_hot_WM = node_df_hot_WM.rename(columns={'pre_syn_count':'out degree','post_syn_count':'in degree'}) #Relabel pre_synaptic_count as out degree and vice versa
#checking for unique cell types to set node attributes 
node_df_WM['subclass'].unique()

array(['5P-PT', '6P-CT', '6P-IT', 'MC', '5P-NP', 'BPC', 'BC'],
      dtype=object)

In [64]:
node_attributes_WM=['in degree','out degree',
        'Cell Type_5P-PT', 'Cell Type_6P-CT', 'Cell Type_6P-IT','Cell Type_5P-NP',
        'Cell Type_BC', 'Cell Type_BPC','Cell Type_MC',
        'Cell Classification_excitatory','Cell Classification_inhibitory']

#set node attributes 
for index, row in node_df_hot_WM.iterrows():
     node_attr_dict_WM={k: float(row.to_dict()[k]) for k in node_attributes_WM}
     G_WM.nodes[row['pt_root_id']].update(node_attr_dict_WM)

#creating py geometric data object
Data_WM=utils.from_networkx(G_WM, group_node_attrs=('in degree','out degree',
        'Cell Type_5P-PT', 'Cell Type_6P-CT', 'Cell Type_6P-IT','Cell Type_5P-NP',
        'Cell Type_BC', 'Cell Type_BPC','Cell Type_MC',
        'Cell Classification_excitatory','Cell Classification_inhibitory'
        ))

In [65]:
#running DOMINANT
model_DOM_WM = DOMINANT()
model_DOM_WM.fit(Data_WM)

labels_DOM_WM = model_DOM_WM.predict(Data_WM)
print('Labels:')
print(labels_DOM_WM)

outlier_scores_DOM_WM = model_DOM_WM.decision_function(Data_WM)
print('Raw scores:')
print(outlier_scores_DOM_WM)

prob_DOM_WM = model_DOM_WM.predict_proba(Data_WM)
print('Probability:')
print(prob_DOM_WM)

label_DOM_WM, confidence_DOM_WM = model_DOM_WM.predict(Data_WM, return_confidence=True)
print('Labels:')
print(labels_DOM_WM)
print('Confidence:')
print(confidence_DOM_WM)

Labels:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Raw scores:
[ 3909043.5       2865150.5       4388877.5       7620809.
  4407542.5       4542720.        7145195.5       3815288.75
  5091525.        3510367.75      8035955.5       4310798.
  4358064.5       5583428.5       8369522.        3332929.
  7109340.5       7268712.5       3830745.75      7309313.
  6414402.5       4063950.25      5298701.        5917345.
  1567520.375     5784848.5       5815908.5       7511555.
 10033554.       10678016.        5301403.5       7208579.5
  4426439.        5836269.5       1620657.875     7559037.
  7143931.        4544480.5       7663163.5       8981232.
 11126715.        592

/Users/vivialung/CRIMSON Coding/madan_test/lib/python3.7/site-packages/pygod/utils/utility.py:52: UserWarning: The cuda is not available. Set to cpu.
  warnings.warn('The cuda is not available. Set to cpu.')


In [66]:
print(G_WM)

DiGraph with 180 nodes and 454 edges


In [68]:
#number of anomalous nodes labeled by DOMINANT
np.sum(labels_DOM_WM)

13

In [126]:
#Grabbing seg_ids of anomalous nodes labeled by DOMINANT
DOMINANT_WM_AD = node_df_WM.copy()
DOMINANT_WM_AD['DOM_labels'] = labels_DOM_WM
DOMINANT_WM_AD = DOMINANT_WM_AD[DOMINANT_WM_AD['DOM_labels'] == 1]

In [128]:
#finding 80th percentile (top 20% of anomalous nodes)
percentile80_WM = np.percentile(prob_DOM_WM[:,1], 80)
WM_AD_indarray = prob_DOM_WM[:,1] > percentile80_WM
DOMINANT_WM_AD_80 = node_df_WM.iloc[WM_AD_indarray]
DOMINANT_WM_AD_80

,id,pt_root_id,pt_position,timestamp,layer,cell_polarity,subclass,pre_syn_count,post_syn_count,unique_pre_syn_target_count,unique_post_syn_target_count
3,470892,864691135695947162,[272928 254640 24485],2022-06-09 14:30:32.002707,WM,excitatory,6P-CT,145.0,1297.0,120.0,1137.0
6,407442,864691135645051375,[234672 253728 18581],2022-06-09 14:30:32.002707,WM,excitatory,6P-IT,126.0,2551.0,92.0,2205.0
10,343716,864691135654019266,[201648 257392 18961],2022-06-09 14:30:32.002707,WM,excitatory,6P-CT,197.0,1747.0,154.0,1569.0
14,132169,864691135181821954,[112208 257792 16298],2022-06-09 14:30:32.002707,WM,excitatory,6P-CT,244.0,1051.0,199.0,870.0
16,275443,864691135926663380,[166992 261696 18437],2022-06-09 14:30:32.002707,WM,excitatory,6P-CT,489.0,2181.0,409.0,1950.0
17,375142,864691135850406343,[222464 247184 15995],2022-06-09 14:30:32.002707,WM,excitatory,6P-IT,51.0,859.0,31.0,778.0
19,679270,864691136237123343,[369744 249728 25927],2022-06-09 14:30:32.002707,WM,excitatory,6P-CT,152.0,995.0,120.0,824.0
27,677789,864691136937056991,[362880 247616 19333],2022-06-09 14:30:32.002707,WM,excitatory,6P-CT,92.0,1542.0,68.0,1376.0
28,437829,864691135447056468,[259728 251936 18611],2022-06-09 14:30:32.002707,WM,excitatory,6P-CT,348.0,1468.0,298.0,1257.0
29,136093,864691135815483779,[109520 264176 18869],2022-06-09 14:30:32.002707,WM,excitatory,6P-IT,106.0,1747.0,92.0,1483.0


In [129]:
#DOMINANT_WM_AD.to_csv('DOMINANT_WM_AD.csv')